## This notebook use the **Mapillary API** to request images and metadata given a set of points, and storing them in locally Database. 

[Mapillary API Documentation](https://www.mapillary.com/developer/api-documentation?locale=pt_PT)

#### Import the necessary libraries

In [1]:
# Import library and some pre-installed modules
import os
import sys
from IPython.display import display, Markdown

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [3]:
# Get current working directory
os.getcwd()

'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

In [ ]:
# Import and Reload the modules to ensure any changes are reflected
import importlib

import src.mapillary_api as mapillary_api
import src.mapillary_tile_downloader as mapillary_tile_downloader
import src.mapillary_metadata_enricher as mapillary_metadata_enricher

importlib.reload(mapillary_api)
importlib.reload(mapillary_tile_downloader)
importlib.reload(mapillary_metadata_enricher)

<module 'src.mapillary_metadata_enricher' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/src/mapillary_metadata_enricher.py'>

### Mapillary Coverage Tiles Requests

In [ ]:
from src.mapillary_tile_downloader import (
    ler_token_mapillary,
    processar_area_abrangente,
    salvar_resultados
)
import geopandas as gpd
from shapely.geometry import shape
import time
import json

# Configurações
TOKEN = ler_token_mapillary()
ZOOM = 14 # Nível de zoom para a área de interesse (> 14)
OUTPUT_GPKG = "resultados/mapillary_coverage.gpkg"

# Carregar área de interesse (exemplo: bbox de um GeoJSON)
with open("data/input_code1/limite_bairro_SaoJose.geojson") as f:
    geojson = json.load(f)

# Criar geometria e obter bbox
geom = shape(geojson['features'][0]['geometry'])
bbox = list(geom.bounds)  # [minx, miny, maxx, maxy]

print(f"Bbox simplificada: {bbox}")
print(f"Zoom: {ZOOM}")

# Processar toda a área
start_time = time.time()
gdf_pontos = processar_area_abrangente(bbox, TOKEN, ZOOM)
tempo_processamento = time.time() - start_time

print(f"\nProcessamento concluído em {tempo_processamento:.2f} segundos")
print(f"Total de pontos encontrados: {len(gdf_pontos)}")

# Salvar resultados
if not gdf_pontos.empty:
    salvar_resultados(gdf_pontos, OUTPUT_GPKG)
    
    # Visualizar amostra
    print("\nAmostra dos dados:")
    print(gdf_pontos.head())
else:
    print("Nenhum ponto encontrado na área")

Bbox simplificada: [-43.97143060018652, -19.86330206621314, -43.95855368488308, -19.855735038210998]
Zoom: 14
Processando 2 tiles para a área...

Processamento concluído em 6.40 segundos
Total de pontos encontrados: 65656
Pontos salvos em resultados/mapillary_coverage.gpkg

Amostra dos dados:
           image_id    captured_at  compass_angle       creator_id  \
0  2908770462775291  1621517547798      89.572235  100125502238568   
1  3212192129017619  1623518220500       0.000000  100687885862968   
2   305576207718647  1572548452669     169.347198  102898865287539   
3   946440085908606  1572639932669      90.000001  102898865287539   
4   336907704666026  1621513594799     253.800385  100125502238568   

              sequence_id  is_pano  organization_id  tile_z  tile_x  tile_y  \
0  fp3sbow6ko42fulolmgglw    False     1.805884e+15      14    6190    9114   
1  OaBKRlM01PnF4QuprtIGck     True              NaN      14    6190    9114   
2  h698vuslwdmmgifsubbkx3    False              

In [13]:
gdf_pontos.head()

,image_id,captured_at,compass_angle,creator_id,sequence_id,is_pano,organization_id,tile_z,tile_x,tile_y,geometry
0,2908770462775291,1621517547798,89.572235,100125502238568,fp3sbow6ko42fulolmgglw,False,1.805884e+15,14,6190,9114,POINT (-43.98209 -19.86281)
1,3212192129017619,1623518220500,0.000000,100687885862968,OaBKRlM01PnF4QuprtIGck,True,NaN,14,6190,9114,POINT (-43.98198 -19.86107)
2,305576207718647,1572548452669,169.347198,102898865287539,h698vuslwdmmgifsubbkx3,False,NaN,14,6190,9114,POINT (-43.98188 -19.86711)
3,946440085908606,1572639932669,90.000001,102898865287539,kpy34sp5f8nri7e04vgfvb,False,NaN,14,6190,9114,POINT (-43.98028 -19.84945)
4,336907704666026,1621513594799,253.800385,100125502238568,pbn2ocqev6wmwnyrcxqim7,False,1.805884e+15,14,6190,9114,POINT (-43.97292 -19.86371)


### Enrich the points retrived from Mapillary Coverage tiles with image Metadata

In [17]:
# notebook_enrichment.ipynb
from src.mapillary_metadata_enricher import (
    ler_token_mapillary,
    enriquecer_geodataframe,
    salvar_geodataframe_enriquecido
)
import geopandas as gpd
import time

# 1. Configuração inicial
TOKEN = ler_token_mapillary()
INPUT_GPKG = "resultados/mapillary_coverage.gpkg"
OUTPUT_GPKG = "resultados/mapillary_data_enriched.gpkg"

# 2. Carregar dados existentes
print(f"Carregando dados de {INPUT_GPKG}")
gdf = gpd.read_file(INPUT_GPKG, layer='mapillary_points')
print(f"Total de pontos carregados: {len(gdf)}")

# 3. Enriquecer dados (buscar TODOS os campos disponíveis)
start_time = time.time()
gdf_enriched = enriquecer_geodataframe(
    gdf,
    TOKEN,
    max_workers=15,
    batch_size=50
)
enrichment_time = time.time() - start_time

print(f"\nEnriquecimento concluído em {enrichment_time/60:.2f} minutos")
print(f"Novas colunas adicionadas: {len(gdf_enriched.columns) - len(gdf.columns)}")

# 4. Salvar resultados
salvar_geodataframe_enriquecido(gdf_enriched, OUTPUT_GPKG)

# 5. Verificar resultados
if not gdf_enriched.empty:
    print("\nCampos disponíveis:")
    print(list(gdf_enriched.columns))
    
    # Mostrar campos específicos com datas
    print("\nAmostra de dados com datas:")
    sample = gdf_enriched[['image_id', 'captured_at', 'captured_date']].head(3)
    print(sample)
    
    # Verificar se captured_at original foi preservado
    original_captured = gdf['captured_at'].iloc[0]
    enriched_captured = gdf_enriched['captured_at'].iloc[0]
    print(f"\nComparação captured_at:")
    print(f"Original: {original_captured} | Enriquecido: {enriched_captured}")
    print("Valores coincidem?" , "Sim" if original_captured == enriched_captured else "Não")

Carregando dados de resultados/mapillary_coverage.gpkg


2025-06-18 10:51:40,607 - INFO - Iniciando enriquecimento para 65656 imagens únicas
2025-06-18 10:51:40,608 - INFO - Processando lote 1: IDs 0 a 49


Total de pontos carregados: 65656


2025-06-18 10:51:43,659 - INFO - Processando lote 2: IDs 50 a 99
2025-06-18 10:51:46,932 - INFO - Processando lote 3: IDs 100 a 149
2025-06-18 10:51:49,731 - INFO - Processando lote 4: IDs 150 a 199
2025-06-18 10:51:52,561 - INFO - Processando lote 5: IDs 200 a 249
2025-06-18 10:51:55,299 - INFO - Processando lote 6: IDs 250 a 299
2025-06-18 10:51:57,985 - INFO - Processando lote 7: IDs 300 a 349
2025-06-18 10:52:00,723 - INFO - Processando lote 8: IDs 350 a 399
2025-06-18 10:52:03,432 - INFO - Processando lote 9: IDs 400 a 449
2025-06-18 10:52:06,060 - INFO - Processando lote 10: IDs 450 a 499
2025-06-18 10:52:08,490 - INFO - Processando lote 11: IDs 500 a 549
2025-06-18 10:52:10,960 - INFO - Processando lote 12: IDs 550 a 599
2025-06-18 10:52:13,410 - INFO - Processando lote 13: IDs 600 a 649
2025-06-18 10:52:15,943 - INFO - Processando lote 14: IDs 650 a 699
2025-06-18 10:52:18,388 - INFO - Processando lote 15: IDs 700 a 749
2025-06-18 10:52:20,923 - INFO - Processando lote 16: IDs 

ValueError: Incompatible indexer with Series

In [43]:
# Test toread the Region of Interest (ROI)
# Mappillary API requires a ROI with a bounding box
sample_bbox = gdf_roi.geometry[0].bounds
sample_bbox

(-42.87785750351895,
 -20.75626257499658,
 -42.8768940754271,
 -20.755361674095678)

In [44]:
# using the basic function to fetch mappillary data:
mapillary_md_gdf = process_mapillary_data(gdf_roi, base_path)

Processado case1
Processado case2
Processado case3
Processado case4
Processado case5
Processado case6
Processado case7
Processado case8
Processado case9
Processado case10
Processado case11
Processado case12
Processado case13
Processado case14


In [45]:
# look at the data!
mapillary_md_gdf.head()

,altitude,atomic_scale,camera_parameters,camera_type,captured_at,compass_angle,computed_altitude,computed_compass_angle,computed_geometry,computed_rotation,...,geometry,height,make,model,thumb_original_url,merge_cc,sequence,width,id,study_case
0,672.599976,1.023902,"[0.67907960699744, 0.0058142864967727, -0.0146...",perspective,2021-01-04 16:45:12,211.710800,0.452590,222.410246,"{'type': 'Point', 'coordinates': [-42.87745664...","[0.85236445764161, 1.6509482326972, -2.0259207...",...,POINT (-42.87745 -20.75714),2250,motorola,moto g(8) plus,https://scontent.fsdu2-2.fna.fbcdn.net/m1/v/t6...,4.530633e+17,95mr1kvsla8dr6v83gc7ps,4000,303510574723248,case14
1,0.000000,1.030234,"[0.69383369017713, 0.050212051536499, -0.04336...",perspective,2024-01-21 16:36:59,-15.340940,8.054201,320.167393,"{'type': 'Point', 'coordinates': [-42.87735500...","[1.4319326179478, 0.60896395573499, -0.4647242...",...,POINT (-42.87725 -20.75694),3024,samsung,SM-M526B,https://scontent.fsdu2-2.fna.fbcdn.net/m1/v/t6...,8.275347e+07,X9IAV8EcNQqwszOtFaulPR,4032,1019390623176236,case14
2,669.358678,0.975600,"[0.68307899068399, 0.019451426917106, -0.03199...",perspective,2021-05-28 16:44:22,112.718292,4.154441,127.331356,"{'type': 'Point', 'coordinates': [-42.87724935...","[0.74968964681063, -1.4973818669163, 1.7945128...",...,POINT (-42.87718 -20.75731),2250,motorola,moto g(8) plus,https://scontent.fsdu2-2.fna.fbcdn.net/m1/v/t6...,9.731326e+08,ir95o7nh5rjnopjk4shv6b,4000,1146349645829232,case14
3,0.000000,1.028763,"[0.69383369017713, 0.050212051536499, -0.04336...",perspective,2024-01-21 16:36:49,291.658978,12.474509,241.763720,"{'type': 'Point', 'coordinates': [-42.87694023...","[0.85209459677827, 1.4905008394785, -1.5881611...",...,POINT (-42.87689 -20.75687),3024,samsung,SM-M526B,https://scontent.fsdu2-2.fna.fbcdn.net/m1/v/t6...,8.275347e+07,X9IAV8EcNQqwszOtFaulPR,4032,704128555155186,case14
4,0.000000,0.991700,"[0.64884811450484, 0.0044671507507445, -6.2396...",perspective,2022-05-25 13:19:50,0.000000,3.756752,34.421759,"{'type': 'Point', 'coordinates': [-42.87752967...","[1.5448218035727, -0.42299975693483, 0.5207972...",...,POINT (-42.87749 -20.75723),2160,samsung,SM-M526B,https://scontent.fsdu2-2.fna.fbcdn.net/m1/v/t6...,1.384610e+09,RvdMcwxGeLKsHyC3U79hkm,3840,735978867534538,case14


In [46]:
len(mapillary_md_gdf)

246

In [47]:
mapillary_md_gdf.columns

Index(['altitude', 'atomic_scale', 'camera_parameters', 'camera_type',
       'captured_at', 'compass_angle', 'computed_altitude',
       'computed_compass_angle', 'computed_geometry', 'computed_rotation',
       'creator', 'exif_orientation', 'geometry', 'height', 'make', 'model',
       'thumb_original_url', 'merge_cc', 'sequence', 'width', 'id',
       'study_case'],
      dtype='object')

In [ ]:
# Save the entire mapillary_md_gdf as GeoJSON
# Definir o caminho de saída completo, incluindo o nome do arquivo e a extensão .geojson
output_path = 'sli_roi/napillary/mapillary_md_gdf.geojson'

# Verificar se o GeoDataFrame não está vazio
if not mapillary_md_gdf.empty:
    # Salvar o GeoDataFrame como GeoJSON
    mapillary_md_gdf.to_file(output_path, driver='GeoJSON')
    print(f"GeoDataFrame salvo em: {output_path}")
else:
    print("GeoDataFrame está vazio e não foi salvo.")

In [ ]:
# Definir o caminho de entrada
file_path2 = 'sli_roi/napillary/mapillary_md_gdf.geojson'

# Chamar a função
mapillary_md_gdf_filtered = load_geojson(file_path2)

In [49]:
len(mapillary_md_gdf_filtered)

1296

### Retrieve and download the Mapillary images

In [ ]:
# download all the images in the ROI

outfolderpath = 'sli_roi/napillary'

download_all_pictures_from_gdf(mapillary_md_gdf_filtered,outfolderpath)

In [ ]:
# Test to download in the vicinity of a Point:
buffer = 20

#p = Point(-49.27950,-25.45297)
p = Point(-42.877416, -20.755756)
buffer = radius_to_degrees(buffer, p.y)
p_bounds = p.buffer(buffer).bounds

# using the basic function to fetch mappillary data:
p_metadata = get_mapillary_images_metadata(*p_bounds,outfolderpath)
p_md_gdf = mapillary_data_to_gdf(p_metadata,outfolderpath)
download_all_pictures_from_gdf(p_md_gdf,outfolderpath)


0it [00:00, ?it/s]


In [4]:
# scan all the images in the folder
outfolderpath = './sli_roi/mapillary'
output_csv_path = './sli_roi/mapillary/image_paths.csv'
scan_images_and_save_to_csv(outfolderpath, output_csv_path)

CSV salvo em: ./sli_roi/mapillary/image_paths.csv
